In [ ]:
import tensorflow as tf
import glob
import os
from keras import Input
import cv2
import numpy as np
import time
from keras.applications import VGG19
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization, Activation, LeakyReLU, Add, Dense, PReLU, Flatten
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.models import Model
from keras.optimizers import Adam
from keras_preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
from PIL import Image
import torchvision

In [ ]:
residual_blocks = 16
momentum =0.8 
input_shape = (64,64,3)

In [ ]:
def residual_block(x):
  '''
  Defining residual block
  '''
  filters = [64, 64]
  kernel_size = 3
  strides = 1
  padding = "same"
  momentum = 0.8
  activation = "relu"
  res = Conv2D(filters=filters[0], kernel_size=kernel_size,
  strides=strides, padding=padding)(x)
  res = Activation(activation=activation)(res)
  res = BatchNormalization(momentum=momentum)(res)
  res = Conv2D(filters=filters[1], kernel_size=kernel_size,
  strides=strides, padding=padding)(res)
  res = BatchNormalization(momentum=momentum)(res)
  # Add res and x
  res = Add()([res, x])
  return res


In [ ]:
def build_generator():

 """
 Create a generator network using the hyperparameter values defined below
 :return:
 """
 residual_blocks = 16
 momentum = 0.8
 input_shape = (64, 64, 3)
# Input Layer of the generator network
 input_layer = Input(shape=input_shape)
# Add the pre-residual block
 gen1 = Conv2D(filters=64, kernel_size=9, strides=1, padding='same',
 activation='relu')(input_layer)
  # Add 16 residual blocks
 res = residual_block(gen1)
 for i in range(residual_blocks - 1):
   res = residual_block(res)
  # Add the post-residual block
 gen2 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(res)
 gen2 = BatchNormalization(momentum=momentum)(gen2)
  # Take the sum of the output from the pre-residual block(gen1) and the post-residual block(gen2)
 gen3 = Add()([gen2, gen1])
  # Add an upsampling block
 gen4 = UpSampling2D(size=2)(gen3)
 gen4 = Conv2D(filters=256, kernel_size=3, strides=1, padding='same')(gen4)
 gen4 = Activation('relu')(gen4)
  # Add another upsampling block
 gen5 = UpSampling2D(size=2)(gen4)
 gen5 = Conv2D(filters=256, kernel_size=3, strides=1,
 padding='same')(gen5)
 gen5 = Activation('relu')(gen5)
# Output convolution layer
 gen6 = Conv2D(filters=3, kernel_size=9, strides=1, padding='same')(gen5)
 output = Activation('tanh')(gen6)
# Keras model
 model = Model(inputs=[input_layer], outputs=[output],
 name='generator')
 return model
  

Discriminator

In [ ]:
def build_discriminator():
 """
 Create a discriminator network using the hyperparameter values defined below
 :return:
 """

 leakyrelu_alpha = 0.2
 input_shape = (256,256,3)


# Firrst input llayer for Disc

 input_layer = Input(shape=input_shape)

# Add convolution blocks
 dis1 = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
 dis1 = LeakyReLU(alpha=leakyrelu_alpha)(dis1)


# Seven inner blocks for DIsc

 dis2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(dis1)
 dis2 = LeakyReLU(alpha=leakyrelu_alpha)(dis2)
 dis2 = BatchNormalization(momentum=momentum)(dis2)
# Add the third convolution block
 dis3 = Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(dis2)
 dis3 = LeakyReLU(alpha=leakyrelu_alpha)(dis3)
 dis3 = BatchNormalization(momentum=momentum)(dis3)
# Add the fourth convolution block
 dis4 = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(dis3)
 dis4 = LeakyReLU(alpha=leakyrelu_alpha)(dis4)
 dis4 = BatchNormalization(momentum=0.8)(dis4)
# Add the fifth convolution block
 dis5 = Conv2D(256, kernel_size=3, strides=1, padding='same')(dis4)
 dis5 = LeakyReLU(alpha=leakyrelu_alpha)(dis5)
 dis5 = BatchNormalization(momentum=momentum)(dis5)
# Add the sixth convolution block
 dis6 = Conv2D(filters=256, kernel_size=3, strides=2, padding='same')(dis5)
 dis6 = LeakyReLU(alpha=leakyrelu_alpha)(dis6)
 dis6 = BatchNormalization(momentum=momentum)(dis6)
# Add the seventh convolution block
 dis7 = Conv2D(filters=512, kernel_size=3, strides=1, padding='same')(dis6)
 dis7 = LeakyReLU(alpha=leakyrelu_alpha)(dis7)
 dis7 = BatchNormalization(momentum=momentum)(dis7)
# Add the eight convolution block
 dis8 = Conv2D(filters=512, kernel_size=3, strides=2, padding='same')(dis7)
 dis8 = LeakyReLU(alpha=leakyrelu_alpha)(dis8)
 dis8 = BatchNormalization(momentum=momentum)(dis8)


# Dense layer with 1024 neurons 

 dis9 = Dense(units=1024)(dis8)
 dis9 = LeakyReLU(alpha=0.2)(dis9)

# Last dense layer with sigmoid activation for prediction
 output = Dense(units=1, activation='sigmoid')(dis9)
 # call model
 model = Model(inputs=[input_layer], outputs=[output], name='discriminator')
 return model

In [ ]:
# Load and extract feature using VGG19 Network

def build_vgg():
 """
 Build the VGG network to extract image features
 """
 input_shape = (256, 256, 3)
 # Load a pre-trained VGG19 model trained on 'Imagenet' dataset
 vgg = VGG19(weights="imagenet", include_top=False)
 input_layer = Input(shape=input_shape)
 vgg.outputs = [vgg.layers[9].output]
 # Extract features
 features = vgg(input_layer)
 # Create a Keras model
 model = Model(inputs=[input_layer], outputs=[features])
 return model

In [ ]:
# Adversarial network for loss

def build_adversarial_model(generator, discriminator, vgg):
 input_low_resolution = Input(shape=(64, 64, 3))
 fake_hr_images = generator(input_low_resolution)
 fake_features = vgg(fake_hr_images)
 discriminator.trainable = False
 output = discriminator(fake_hr_images)
 model = Model(inputs=[input_low_resolution],
 outputs=[output, fake_features])
 for layer in model.layers:
  print(layer.name, layer.trainable)
  print(model.summary())
 return model

Training

In [ ]:
data_dir = "/content/img_align_celeba/*.*"
epochs = 20000
batch_size = 1
# Shape of low-resolution and high-resolution images
low_resolution_shape = (64, 64, 3)
high_resolution_shape = (256, 256, 3)
common_optimizer = Adam(0.0002, 0.5)

In [ ]:
# Build and compile VGG19 network
vgg = VGG19(weights="imagenet",include_top=False, input_shape=(256, 256, 3))
model = Model(inputs=vgg.inputs, outputs=vgg.layers[9].output)
vgg = model
vgg.trainable = False
vgg.compile(loss='mse', optimizer=common_optimizer, metrics=
 ['accuracy'])


# bUild and compile discriminator network

discriminator = build_discriminator()
discriminator.compile(loss='mse', optimizer=common_optimizer,
 metrics=['accuracy'])


# build generator

generator = build_generator()

# create adversarial model by creating two layers

input_high_resolution = Input(shape=high_resolution_shape)
input_low_resolution = Input(shape=low_resolution_shape)


# Generate high resolution using Generator
generated_high_resolution_images = generator(input_low_resolution)

# Use VGG19 to extaract the features
features = vgg(generated_high_resolution_images)

# make discriminator non  trainable
discriminator.trainable = False

# Generator prediction on the fake images
probs = discriminator(generated_high_resolution_images)

# Compile adversarial network

adversarial_model = Model([input_low_resolution, input_high_resolution], [probs, features])
adversarial_model.compile(loss=['binary_crossentropy', 'mse'], loss_weights=[1e-3, 1], optimizer=common_optimizer)

# Visualize using tensorboard

tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
tensorboard.set_model(generator)
tensorboard.set_model(discriminator)



            
  

In [ ]:
# Code to generate low and high resolution images

def sample_images(data_dir, batch_size, high_resolution_shape, low_resolution_shape):


# Make a list of all images inside the data directory
  all_images = glob.glob(data_dir)
  # Choose a random batch of images
  images_batch = np.random.choice(all_images, size=batch_size)
  low_resolution_images = []
  high_resolution_images = []
  for img in images_batch:
  # Get an ndarray of the current image
    img1 = cv2.imread(img)
    img1 = np.asarray(img1)
    img1 = img1.astype(np.float32)
  # Resize the image
  from skimage.transform import resize                    
  
  img1_high_resolution = resize(img1, high_resolution_shape)
  img1_low_resolution = resize(img1, low_resolution_shape)
  # Do a random flip
  if np.random.random() < 0.5:
    img1_high_resolution = np.fliplr(img1_high_resolution)
    img1_low_resolution = np.fliplr(img1_low_resolution)
  high_resolution_images.append(img1_high_resolution)
  low_resolution_images.append(img1_low_resolution)
  return np.array(high_resolution_images),np.array(low_resolution_images)


Starts Epochs

In [ ]:
for epoch in range(epochs):
 print("Epoch:{}".format(epoch))
 high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir,
 batch_size=batch_size,low_resolution_shape=low_resolution_shape,
 high_resolution_shape=high_resolution_shape)

# Code to generate low and high resolution images

 
 #print(high_resolution_images.shape, low_resolution_images.shape)
 # normalization

 high_resolution_images = high_resolution_images / 127.5 - 1.
 low_resolution_images = low_resolution_images / 127.5 - 1.

 generated_high_resolution_images = generator.predict(low_resolution_images)

 real_labels = np.ones((batch_size, 16, 16, 1))
 fake_labels = np.zeros((batch_size, 16, 16, 1))

 d_loss_real = discriminator.train_on_batch(high_resolution_images, real_labels)
 d_loss_fake = discriminator.train_on_batch(generated_high_resolution_images, fake_labels)

 d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

 high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,high_resolution_shape=high_resolution_shape,low_resolution_shape=low_resolution_shape)

 # Normalize images
 high_resolution_images = high_resolution_images / 127.5 - 1.
 low_resolution_images = low_resolution_images / 127.5 - 1.
 image_features = vgg.predict(high_resolution_images)

 g_loss = adversarial_model.train_on_batch([low_resolution_images, high_resolution_images], [real_labels, image_features])
 #write_log(tensorboard, 'g_loss', g_loss[0], epoch)
 #write_log(tensorboard, 'd_loss', d_loss[0], epoch)




 if epoch % 100 == 0:
  high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size = batch_size,
  high_resolution_shape=high_resolution_shape,low_resolution_shape=low_resolution_shape)
  # Normalize images
  high_resolution_images = high_resolution_images / 127.5 - 1.
  low_resolution_images = low_resolution_images / 127.5 - 1.

  # Generate fake high-resolution images
  generated_images = generator.predict_on_batch(low_resolution_images)

 # Save
  for index, img in enumerate(generated_images):
    plt.imshow(high_resolution_images[index])
    plt.show()
    plt.imshow(low_resolution_images[index])
    plt.show()
    plt.imshow(generated_images[index])
    plt.show()
    #torchvision.utils.save_image(high_resolution_images[index], "/content/images_saved/" + str(index))
  #  save_images(low_resolution_images[index], high_resolution_images[index], img, path="results/img_{}_{}".format(epoch, index))
if cur_step % display_step == 0 and cur_step > 0:
  print('Step {}: SRResNet loss: {:.5f}'.format(cur_step, mean_loss))



